## Exercise 03 - Aggregations

## Imports

In [59]:
import pandas as pd
import sqlite3


## 1. Connect to database and inspect test table

In [60]:
conn = sqlite3.connect('../data/checking-logs.sqlite')

test_schema = pd.io.sql.read_sql('PRAGMA table_info(test);', conn)
test_schema


,cid,name,type,notnull,dflt_value,pk
0,0,uid,TEXT,0,None,0
1,1,labname,TEXT,0,None,0
2,2,first_commit_ts,TIMESTAMP,0,None,0
3,3,first_view_ts,TIMESTAMP,0,None,0


In [61]:
test_preview = pd.io.sql.read_sql(
    'SELECT * FROM test LIMIT 10;',
    conn,
    parse_dates=['first_commit_ts', 'first_view_ts']
)
test_preview


,uid,labname,first_commit_ts,first_view_ts
0,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
1,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
2,user_30,laba04s,2020-04-18 14:51:37.498399,2020-04-17 22:46:26.785035
3,user_14,laba04,2020-04-18 15:14:00.312338,2020-04-18 10:53:52.623447
4,user_14,laba04s,2020-04-18 22:30:30.247628,2020-04-18 10:53:52.623447
5,user_19,laba04,2020-04-20 19:05:01.297780,2020-04-21 20:30:38.034966
6,user_25,laba04,2020-04-20 19:16:50.673054,2020-05-09 23:54:54.260791
7,user_21,laba04,2020-04-21 17:48:00.487806,2020-04-22 22:40:36.824081
8,user_30,project1,2020-04-22 12:36:24.053518,2020-04-17 22:46:26.785035
9,user_21,laba04s,2020-04-22 20:09:21.857747,2020-04-22 22:40:36.824081


## 2. Inspect deadlines table

In [62]:
deadlines_schema = pd.io.sql.read_sql('PRAGMA table_info(deadlines);', conn)
deadlines_schema

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,labs,TEXT,0,None,0
2,2,deadlines,INTEGER,0,None,0


In [63]:
deadlines_preview = pd.io.sql.read_sql('SELECT * FROM deadlines LIMIT 10;', conn)
deadlines_preview

,index,labs,deadlines
0,0,laba04,1587945599
1,1,laba04s,1587945599
2,2,laba05,1588550399
3,4,laba06,1590364799
4,5,laba06s,1590364799
5,3,project1,1589673599


## 3. Delta aggregations vs deadlines

In [64]:
df_min = pd.io.sql.read_sql(
    """
    WITH diffs AS (
        SELECT
            t.uid,
            (JULIANDAY(datetime(d.deadlines, 'unixepoch')) - JULIANDAY(t.first_commit_ts)) * 24 AS diff_hours
        FROM test t
        JOIN deadlines d ON t.labname = d.labs
        WHERE t.labname != 'project1'
    )
    SELECT uid, diff_hours AS min_diff
    FROM diffs
    ORDER BY diff_hours ASC
    LIMIT 1
    """,
    conn
)
df_min

,uid,min_diff
0,user_25,2.867236


In [65]:
df_max = pd.io.sql.read_sql(
    """
    WITH diffs AS (
        SELECT
            t.uid,
            (JULIANDAY(datetime(d.deadlines, 'unixepoch')) - JULIANDAY(t.first_commit_ts)) * 24 AS diff_hours
        FROM test t
        JOIN deadlines d ON t.labname = d.labs
        WHERE t.labname != 'project1'
    )
    SELECT uid, diff_hours AS max_diff
    FROM diffs
    ORDER BY diff_hours DESC
    LIMIT 1
    """,
    conn
)
df_max

,uid,max_diff
0,user_30,202.38473


In [66]:
df_avg = pd.io.sql.read_sql(
    """
    WITH diffs AS (
        SELECT
            (JULIANDAY(datetime(d.deadlines, 'unixepoch')) - JULIANDAY(t.first_commit_ts)) * 24 AS diff_hours
        FROM test t
        JOIN deadlines d ON t.labname = d.labs
        WHERE t.labname != 'project1'
    )
    SELECT AVG(diff_hours) AS avg_diff
    FROM diffs
    """,
    conn
)
df_avg

,avg_diff
0,89.687686


## 4. Avg diff vs pageviews correlation

In [67]:
views_diff = pd.io.sql.read_sql(
    """
    WITH diffs AS (
        SELECT
            t.uid,
            (JULIANDAY(datetime(d.deadlines, 'unixepoch')) - JULIANDAY(t.first_commit_ts)) * 24 AS diff_hours
        FROM test t
        JOIN deadlines d ON t.labname = d.labs
        WHERE t.labname != 'project1'
    ),
    pageviews_count AS (
        SELECT uid, COUNT(*) AS pageviews
        FROM pageviews
        WHERE uid LIKE 'user_%'
        GROUP BY uid
    )
    SELECT
        diffs.uid,
        AVG(diffs.diff_hours) AS avg_diff,
        COALESCE(p.pageviews, 0) AS pageviews
    FROM diffs
    LEFT JOIN pageviews_count p ON diffs.uid = p.uid
    GROUP BY diffs.uid
    ORDER BY diffs.uid
    """,
    conn
)
views_diff_corr = views_diff[['avg_diff', 'pageviews']].corr()


print("--- Views vs Diff Table ---")
print(views_diff.head())

corr_matrix = views_diff[['avg_diff', 'pageviews']].corr()

print("\n--- Correlation Coefficient ---")
print(corr_matrix)

print(f"\nExact Correlation: {corr_matrix.loc['avg_diff', 'pageviews']}")

--- Views vs Diff Table ---
       uid    avg_diff  pageviews
0   user_1   65.119644         28
1  user_10   75.242310         89
2  user_14  159.568696        143
3  user_17   62.207513         47
4  user_18    6.367907          3

--- Correlation Coefficient ---
           avg_diff  pageviews
avg_diff   1.000000   0.279143
pageviews  0.279143   1.000000

Exact Correlation: 0.2791430910925191


## 4. Close connection

In [68]:
conn.close()
